In [1]:
from pycocotools.coco import COCO
import numpy as np
import matplotlib.pyplot as plt

In [2]:
dataType='val2017'
annFile=f'dataset/annotations/instances_{dataType}.json'
coco=COCO(annFile)

loading annotations into memory...
Done (t=1.42s)
creating index...
index created!


In [3]:
catNms = ['airplane', 'cat', 'dog']
catIds = coco.getCatIds(catNms=catNms)
cat_dict = dict(zip(catIds,catNms))

In [4]:
cat_ids = {}
x_id = []
Y = []
total = 0
for cat in catIds:
  cat_ids[cat_dict[cat]] = coco.getImgIds(catIds=cat)
  for img_id in cat_ids[cat_dict[cat]]:
        x_id.append(img_id)
        Y.append(cat_dict[cat])
  print(cat_dict[cat], len(cat_ids[cat_dict[cat]]))
  total += len(cat_ids[cat_dict[cat]])
print(f'\ntotal = {len(catIds)}\ntotal no. of imgs = {total}')

airplane 97
cat 184
dog 177

total = 3
total no. of imgs = 458


In [5]:
from skimage.transform import resize as imresize
import skimage.io as io
import os

def load_images(img_ids, img_dir, img_size, preprocessor):
    x = []
    for img_id in img_ids:
        img_file_name = coco.loadImgs(img_id)[0]['file_name']
        img_complete_path = os.path.join(img_dir,img_file_name)
        img = io.imread(img_complete_path)
        img = imresize(img, (img_size,img_size,3), mode='reflect')
#         img = preprocessor(img)
        x.append(img)
    return x

In [6]:
# import tensorflow.keras as keras
# from tensorflow.keras.applications import inception_v3
# from tensorflow.keras.models import Model
import  keras
from keras.applications import inception_v3
from keras.models import Model
# from tensorflow.keras.applications.imagenet_utils import decode_predictions

img_dir = 'dataset/val2017/'
img_size = 299
preprocessor = inception_v3.preprocess_input
X = load_images(x_id, img_dir, img_size, preprocessor)
X = np.array(X)
print(X.shape)

Using TensorFlow backend.
/home/ubuntu/anaconda3/envs/jo/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


(458, 299, 299, 3)


In [7]:
### loading the model
inception_model = inception_v3.InceptionV3(weights="imagenet", include_top=True)

Instructions for updating:
Colocations handled automatically by placer.


In [8]:
import shap
from skimage.segmentation import quickshift, slic

In [9]:
random_seed = 1
def segmenter(img):
#     return quickshift(img, ratio=0.2, kernel_size=4, max_dist=200,
#                        return_tree=False, random_seed=random_seed)
    return slic(img, n_segments=50, compactness=30, sigma=3)
def mask_image(zs, segmentation, image, background=None):
    if background is None:
        background = image.mean((0,1))
    out = np.zeros((zs.shape[0], image.shape[0], image.shape[1], image.shape[2]))
    for i in range(zs.shape[0]):
        out[i,:,:,:] = image
        for j in range(zs.shape[1]):
            if zs[i,j] == 0:
                out[i][segmentation == j,:] = background
    return out

In [26]:
img_orig = X[0]
segments_slic = segmenter(img_orig)
def f(z):
#     return inception_model.predict(preprocessor(mask_image(z, segments_slic, img_orig, 255)))
    return inception_model.predict(mask_image(z, segments_slic, img_orig, 255))[:2]

In [27]:
explainer = shap.KernelExplainer(f, np.zeros((1,50)))
shap_values = explainer.shap_values(np.ones((1,50)), nsamples=50)



  0%|          | 0/1 [00:00<?, ?it/s]

ValueError: cannot reshape array of size 2000 into shape (50,1000)

In [19]:
len(shap_values)

1000

In [ ]:
from lime.wrappers.scikit_image import SegmentationAlgorithm
random_seed = 1
segmenter = SegmentationAlgorithm('quickshift', kernel_size=4, max_dist=200,
                                  ratio=0.2, random_seed=random_seed)

budget = 50
no_superpixels = 5
def bulk_exp(explainer, images):
    return [explainer.explain_instance(image, inception_model.predict, top_labels=5,
                                        num_features=budget, hide_color=0, num_samples=1000,
                                        segmentation_fn = segmenter)   for image in images]

def get_imgs_from_exps(exps):
    return np.array([exp.get_image_and_mask(exp.top_labels[0], positive_only=True,
                                            num_features=no_superpixels, hide_rest=True)[0] for exp in exps])

In [ ]:
%time exps1 = bulk_exp(explainer, X)
%time exps2 = bulk_exp(explainer, X)

In [ ]:
import pickle
with open('exp1.pkl', 'wb') as f:
    pickle.dump(exps1, f)
with open('exp2.pkl', 'wb') as f:
    pickle.dump(exps2, f)